<a href="https://colab.research.google.com/github/axiom19/Langchain-for-LLM-App-Development/blob/main/Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain for LLM Application Development
<hr>

## Chains


In [2]:
# !pip install openai langchain
# !pip install -qU langchain-openai

In [52]:
import os

# os.environ["OPENAI_API_KEY"]

In [5]:
# loading the data from csv file

import pandas as pd
df = pd.read_csv("Data.csv")

df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [6]:
# import langchain

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
# set up the LLM

llm = ChatOpenAI(temperature=0.9)

In [8]:
# set up the prompt template
prompt = ChatPromptTemplate.from_template(
    template="What is the best name to describe a company that makes {product}?"
)

### LLMChain
<hr>
LLMChain is the most basic type of chain

In [10]:
# set up the chain
chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-10-bddefa0a5dee>:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [14]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Comfort Linens"'

In [15]:
product2 = "Natural Indigo Colored Fabric"
chain.run(product2)

'Indigo Naturals'

In [16]:
product3 = "An AI software which makes a fashion marketing photoshoot with professional models and also creates small videos for the same"
chain.run(product3)

'"ModelAI Marketing Solutions"'

### Sequential Chains
<hr>
Sequential chain is another type of chains. The idea is to combine multiple chains where the output of the one chain is the input of the next chain.

There are two types of Sequential Chains:
<li> SimpleSequentialChain : Single input/output
<li> SequentialChain: multiple inputs/outputs

In [17]:
from langchain.chains import SimpleSequentialChain

In [18]:
# prompt template 1
first_prompt = prompt # using the previous prompt

# chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [19]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [21]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                                verbose=True)

In [22]:
overall_simple_chain.run(product)

RoyaltyLinens
Luxury linens fit for royalty, offering exquisite bedding, towels, and table linens for a regal touch to any home decor.

> Finished chain.


'Luxury linens fit for royalty, offering exquisite bedding, towels, and table linens for a regal touch to any home decor.'

In [23]:
overall_simple_chain(product2)

<ipython-input-23-c490c297bdb3>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  overall_simple_chain(product2)


"Indigo Nature"
Indigo Nature is a sustainable beauty company offering organic, eco-friendly skincare products that harness the power of natural ingredients.

> Finished chain.


{'input': 'Natural Indigo Colored Fabric',
 'output': 'Indigo Nature is a sustainable beauty company offering organic, eco-friendly skincare products that harness the power of natural ingredients.'}

SimpleSequentialChain works well when we have a single input and a single output. Otherwise we can use just **SequentialChain**

In [24]:
from langchain.chains import SequentialChain

In [25]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:{Review}"
)

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"
                     )

In [26]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:{English_Review}"
)

# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                     )

In [27]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                       )

In [28]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                    output_key="followup_message"
                    )

In [29]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

In [35]:
review = df.Review[5]
review

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [36]:
overall_chain(review)


> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer found the taste of the product to be underwhelming and questioned whether it was an old batch or counterfeit.',
 'followup_message': "Merci pour votre commentaire honnête. Nous sommes désolés d'apprendre que vous avez trouvé le goût du produit décevant. Nous pouvons vous assurer que nos produits sont frais et authentiques. Cependant, nous apprécions vos préoccupations et nous prendrons des mesures pour nous assurer que nos produits répondent toujours à vos attentes élevées. Merci de nous avoir informés de votre expérience."}

### Router Chain
<hr>
A router chain in LangChain dynamically routes user inputs to the most appropriate sub-chain or model based on the content of the input.

In [37]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [38]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [39]:
# importing other chains that we need
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [40]:
llm = ChatOpenAI(temperature=0)

In [42]:
# we create destination chain where each chain is a LLMChain

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [43]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [45]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [46]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [47]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [48]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

<ipython-input-48-ae9cc652db54>:1: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain,


In [49]:
chain.run("What is black body radiation?")

physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber and emitter of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [51]:
chain.run("How did the Sikh Empire fall?")

History: {'input': 'How did the Sikh Empire fall?'}
> Finished chain.


'The Sikh Empire, also known as the Sikh Kingdom of the Punjab, fell due to a combination of internal conflicts, external invasions, and the changing political landscape of the region.\n\nOne of the main factors that led to the downfall of the Sikh Empire was internal strife and power struggles among the Sikh leaders after the death of Maharaja Ranjit Singh in 1839. The lack of a strong and capable leader to succeed him resulted in a fragmented and weakened empire.\n\nAdditionally, the British East India Company, seeking to expand its influence in the region, took advantage of the internal divisions within the Sikh Empire and launched the First Anglo-Sikh War in 1845. Despite initial Sikh victories, the British ultimately emerged victorious and annexed significant territories of the Sikh Empire.\n\nThe Second Anglo-Sikh War in 1848 further weakened the Sikh Empire, leading to the annexation of the Punjab by the British in 1849. The British established direct control over the region, ef